# Lab 4: Spark Dataframes and SQL (Part III)
Analyzing airline data with Spark SQL

In [5]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Analyzing airline data") \
    .getOrCreate()

### Exploring SQL query options

We start by some static data by using ```sc.parallelize``` to test.

In [1]:
from pyspark.sql.types import Row
from datetime import datetime

#### Creating a dataframe with different data types

In [23]:
record = sc.parallelize([Row(id = 1,
                             name = "Jill",
                             active = True,
                             clubs = ['chess', 'hockey'],
                             subjects = {"math": 80, 'english': 56},
                             enrolled = datetime(2014, 8, 1, 14, 1, 5)),
                         Row(id = 2,
                             name = "George",
                             active = False,
                             clubs = ['chess', 'soccer'],
                             subjects = {"math": 60, 'english': 96},
                             enrolled = datetime(2015, 3, 21, 8, 2, 5))
])

In [24]:
record_df = record.toDF()
record_df.show()

+------+---------------+-------------------+---+------+--------------------+
|active|          clubs|           enrolled| id|  name|            subjects|
+------+---------------+-------------------+---+------+--------------------+
|  true|[chess, hockey]|2014-08-01 14:01:05|  1|  Jill|[english -> 56, m...|
| false|[chess, soccer]|2015-03-21 08:02:05|  2|George|[english -> 96, m...|
+------+---------------+-------------------+---+------+--------------------+



#### Register the dataframe as a temporary view

* The view is valid for one session
* This is required to run SQL commands on the dataframe

In [25]:
record_df.createOrReplaceTempView("records")

In [26]:
all_records_df = sqlContext.sql('SELECT * FROM records')

all_records_df.show()

+------+---------------+-------------------+---+------+--------------------+
|active|          clubs|           enrolled| id|  name|            subjects|
+------+---------------+-------------------+---+------+--------------------+
|  true|[chess, hockey]|2014-08-01 14:01:05|  1|  Jill|[english -> 56, m...|
| false|[chess, soccer]|2015-03-21 08:02:05|  2|George|[english -> 96, m...|
+------+---------------+-------------------+---+------+--------------------+



We can access array and dictionary (map) elements in the dataframe in SQL

In [27]:
sqlContext.sql('SELECT id, clubs[1], subjects["english"] FROM records').show()

+---+--------+-----------------+
| id|clubs[1]|subjects[english]|
+---+--------+-----------------+
|  1|  hockey|               56|
|  2|  soccer|               96|
+---+--------+-----------------+



We can apply expressions on columns, e.g., negate the active column

In [28]:
sqlContext.sql('SELECT id, NOT active FROM records').show()

+---+------------+
| id|(NOT active)|
+---+------------+
|  1|       false|
|  2|        true|
+---+------------+



### Conditional statements in SQL 

In [29]:
sqlContext.sql('SELECT * FROM records where active').show()

+------+---------------+-------------------+---+----+--------------------+
|active|          clubs|           enrolled| id|name|            subjects|
+------+---------------+-------------------+---+----+--------------------+
|  true|[chess, hockey]|2014-08-01 14:01:05|  1|Jill|[english -> 56, m...|
+------+---------------+-------------------+---+----+--------------------+



In [30]:
sqlContext.sql('SELECT * FROM records where subjects["english"] > 90').show()

+------+---------------+-------------------+---+------+--------------------+
|active|          clubs|           enrolled| id|  name|            subjects|
+------+---------------+-------------------+---+------+--------------------+
| false|[chess, soccer]|2015-03-21 08:02:05|  2|George|[english -> 96, m...|
+------+---------------+-------------------+---+------+--------------------+



#### Global temporary view

* Temporary view shared across multiple sessions
* Kept alive till the Spark application terminates

In [32]:
record_df.createGlobalTempView("global_records")

In [35]:
sqlContext.sql('SELECT * FROM global_temp.global_records').show()

+------+---------------+-------------------+---+------+--------------------+
|active|          clubs|           enrolled| id|  name|            subjects|
+------+---------------+-------------------+---+------+--------------------+
|  true|[chess, hockey]|2014-08-01 14:01:05|  1|  Jill|[english -> 56, m...|
| false|[chess, soccer]|2015-03-21 08:02:05|  2|George|[english -> 96, m...|
+------+---------------+-------------------+---+------+--------------------+



Now, let's play with real data. we start creating a session and load the data. 
We have three data sets, two small and one large.

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Analyzing airline data") \
    .getOrCreate()

In [2]:
from pyspark.sql.types import Row
from datetime import datetime

#### Loading in airline data

In [3]:
#You may need to adjust the paths below to match the exact location on your machine.
airlinesPath = "airlines.csv"
flightsPath = "flights.csv"
airportsPath = "airports.csv"

In [4]:
airlines = spark.read\
                .format("csv")\
                .option("header", "true")\
                .load(airlinesPath)

Register as a temporary view

In [5]:
airlines.createOrReplaceTempView("airlines")

In [6]:
airlines = spark.sql("SELECT * FROM airlines")
airlines.columns

['Code', 'Description']

In [7]:
airlines.show(5)

+-----+--------------------+
| Code|         Description|
+-----+--------------------+
|19031|Mackey Internatio...|
|19032|Munz Northern Air...|
|19033|Cochise Airlines ...|
|19034|Golden Gate Airli...|
|19035|  Aeromech Inc.: RZZ|
+-----+--------------------+
only showing top 5 rows



Load flights data

In [9]:
flights = spark.read\
               .format("csv")\
               .option("header", "true")\
               .load(flightsPath)

In [10]:
flights.createOrReplaceTempView("flights")

flights.columns

['date',
 'airlines',
 'flight_number',
 'origin',
 'destination',
 'departure',
 'departure_delay',
 'arrival',
 'arrival_delay',
 'air_time',
 'distance']

In [11]:
flights.show(5)

+----------+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+
|      date|airlines|flight_number|origin|destination|departure|departure_delay|arrival|arrival_delay|air_time|distance|
+----------+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+
|2014-04-01|   19805|            1|   JFK|        LAX|     0854|          -6.00|   1217|         2.00|  355.00| 2475.00|
|2014-04-01|   19805|            2|   LAX|        JFK|     0944|          14.00|   1736|       -29.00|  269.00| 2475.00|
|2014-04-01|   19805|            3|   JFK|        LAX|     1224|          -6.00|   1614|        39.00|  371.00| 2475.00|
|2014-04-01|   19805|            4|   LAX|        JFK|     1240|          25.00|   2028|       -27.00|  264.00| 2475.00|
|2014-04-01|   19805|            5|   DFW|        HNL|     1300|          -5.00|   1650|        15.00|  510.00| 3784.00|
+----------+--------+-----------

#### Counting with dataframes

In [12]:
flights.count(), airlines.count()

(476881, 1579)

#### Counting using SQL

In [13]:
flights_count = spark.sql("SELECT COUNT(*) FROM flights")
airlines_count = spark.sql("SELECT COUNT(*) FROM airlines")

In [14]:
flights_count, airlines_count

(DataFrame[count(1): bigint], DataFrame[count(1): bigint])

Note, that the result of an SQL expression is a dataframe. So, we have to call ```collect()``` to get an array and access the first row/cell.

In [15]:
flights_count.collect()[0][0], airlines_count.collect()[0][0]

(476881, 1579)

#### Dataframes created using SQL commands can be aggregated, grouped etc. exactly as before

In [28]:
total_distance_df = spark.sql("SELECT distance FROM flights")\
                         .agg({"distance":"sum"})\
                         .withColumnRenamed("sum(distance)","total_distance")

In [29]:
total_distance_df.show()

+--------------+
|total_distance|
+--------------+
|  3.79052917E8|
+--------------+



#### Analyzing flight delays

In [16]:
all_delays_2012 = spark.sql(
    "SELECT date, airlines, flight_number, departure_delay " +
    "FROM flights WHERE departure_delay > 0 and year(date) = 2012")

In [17]:
all_delays_2012.show(5)

+----+--------+-------------+---------------+
|date|airlines|flight_number|departure_delay|
+----+--------+-------------+---------------+
+----+--------+-------------+---------------+



In [18]:
all_delays_2014 = spark.sql(
    "SELECT date, airlines, flight_number, departure_delay " +
    "FROM flights WHERE departure_delay > 0 and year(date) = 2014")

all_delays_2014.show(5)

+----------+--------+-------------+---------------+
|      date|airlines|flight_number|departure_delay|
+----------+--------+-------------+---------------+
|2014-04-01|   19805|            2|          14.00|
|2014-04-01|   19805|            4|          25.00|
|2014-04-01|   19805|            6|         126.00|
|2014-04-01|   19805|            7|         125.00|
|2014-04-01|   19805|            8|           4.00|
+----------+--------+-------------+---------------+
only showing top 5 rows



In [19]:
all_delays_2014.createOrReplaceTempView("all_delays")

In [20]:
all_delays_2014.orderBy(all_delays_2014.departure_delay.desc()).show(5)

+----------+--------+-------------+---------------+
|      date|airlines|flight_number|departure_delay|
+----------+--------+-------------+---------------+
|2014-04-27|   20366|         5246|          99.00|
|2014-04-27|   19393|         2948|          99.00|
|2014-04-27|   20366|         5365|          99.00|
|2014-04-26|   19977|          616|          99.00|
|2014-04-27|   20366|         6030|          99.00|
+----------+--------+-------------+---------------+
only showing top 5 rows



#### Total number of delayed flights in 2014

In [21]:
delay_count = spark.sql("SELECT COUNT(departure_delay) FROM all_delays")

In [22]:
delay_count.show()

+----------------------+
|count(departure_delay)|
+----------------------+
|                179015|
+----------------------+



In [59]:
delay_count.collect()[0][0]

179015

#### Percentage of flights delayed

In [61]:
delay_percent = delay_count.collect()[0][0] / flights_count.collect()[0][0] * 100
delay_percent

37.53871510922012

### Finding delay per aIrlines

In [62]:
delay_per_airline = spark.sql("SELECT airlines, departure_delay FROM flights")\
                         .groupBy("airlines")\
                         .agg({"departure_delay":"avg"})\
                         .withColumnRenamed("avg(departure_delay)", "departure_delay")

In [63]:
delay_per_airline.orderBy(delay_per_airline.departure_delay.desc()).show(5)

+--------+------------------+
|airlines|   departure_delay|
+--------+------------------+
|   19393|13.429567657134724|
|   20366|12.296210112379818|
|   19977| 8.818392620527979|
|   20436| 8.716275167785234|
|   20409|  8.31110357194785|
+--------+------------------+
only showing top 5 rows



In [64]:
delay_per_airline.createOrReplaceTempView("delay_per_airline")

In [65]:
delay_per_airline = spark.sql("SELECT * FROM delay_per_airline ORDER BY departure_delay DESC")

In [66]:
delay_per_airline.show(5)

+--------+------------------+
|airlines|   departure_delay|
+--------+------------------+
|   19393|13.429567657134724|
|   20366|12.296210112379818|
|   19977| 8.818392620527979|
|   20436| 8.716275167785234|
|   20409|  8.31110357194785|
+--------+------------------+
only showing top 5 rows



#### SQL join operations 

* Get the names of the delayed flights

In [70]:
delay_per_airline = spark.sql("SELECT * FROM delay_per_airline " +
                              "JOIN airlines ON airlines.code = delay_per_airline.airlines " +
                              "ORDER BY departure_delay DESC")

delay_per_airline.show(5)

+--------+------------------+-----+--------------------+
|airlines|   departure_delay| Code|         Description|
+--------+------------------+-----+--------------------+
|   19393|13.429567657134724|19393|Southwest Airline...|
|   20366|12.296210112379818|20366|ExpressJet Airlin...|
|   19977| 8.818392620527979|19977|United Air Lines ...|
|   20436| 8.716275167785234|20436|Frontier Airlines...|
|   20409|  8.31110357194785|20409| JetBlue Airways: B6|
+--------+------------------+-----+--------------------+
only showing top 5 rows



In [71]:
delay_per_airline.drop("code").show(5)

+--------+------------------+--------------------+
|airlines|   departure_delay|         Description|
+--------+------------------+--------------------+
|   19393|13.429567657134724|Southwest Airline...|
|   20366|12.296210112379818|ExpressJet Airlin...|
|   19977| 8.818392620527979|United Air Lines ...|
|   20436| 8.716275167785234|Frontier Airlines...|
|   20409|  8.31110357194785| JetBlue Airways: B6|
+--------+------------------+--------------------+
only showing top 5 rows



### Window functions

In SQL, window functions allow making computations over a range of rows (tuples). Window functions provide a handy approach to answer analytical queries like *Find the best selling product in each category in last week's sale*. Window functions are part of the SQL standard and they are also available in Spark SQL/Dataframes.

In [23]:
products = spark.read\
                .format("csv")\
                .option("header", "true")\
                .load('products.csv')

In [24]:
products.show()

+----------+--------+-----+
|   product|category|price|
+----------+--------+-----+
|Samsung TX|  Tablet|  999|
|Samsung JX|  Mobile|  799|
|Redmi Note|  Mobile|  399|
|        Mi|  Mobile|  299|
|      iPad|  Tablet|  789|
|    iPhone|  Mobile|  999|
|  Micromax|  Mobile|  249|
|    Lenovo|  Tablet|  499|
|   OnePlus|  Mobile|  356|
|        Xu|  Tablet|  267|
+----------+--------+-----+



#### Window rank function

You can order the data in the range specified and automatically you get a ```rank``` column for each tuple. We need to specify: the column(s) to partition the data with, the column(s) to sort the data with in order to compute the rank.

In [26]:
import sys
from pyspark.sql.window import Window
import pyspark.sql.functions as func

In [27]:
windowSpec1 = Window \
    .partitionBy(products['category']) \
    .orderBy(products['price'].desc())

In [28]:
price_rank = (func.rank().over(windowSpec1))

In [29]:
product_rank = products.select(
        products['product'],
        products['category'],
        products['price']
).withColumn('rank', price_rank)

product_rank.show()

+----------+--------+-----+----+
|   product|category|price|rank|
+----------+--------+-----+----+
|    iPhone|  Mobile|  999|   1|
|Samsung JX|  Mobile|  799|   2|
|Redmi Note|  Mobile|  399|   3|
|   OnePlus|  Mobile|  356|   4|
|        Mi|  Mobile|  299|   5|
|  Micromax|  Mobile|  249|   6|
|Samsung TX|  Tablet|  999|   1|
|      iPad|  Tablet|  789|   2|
|    Lenovo|  Tablet|  499|   3|
|        Xu|  Tablet|  267|   4|
+----------+--------+-----+----+



#### Window max function between rows

In [9]:
windowSpec2 = Window \
    .partitionBy(products['category']) \
    .orderBy(products['price'].desc()) \
    .rowsBetween(-1, 0)

In [10]:
price_max = (func.max(products['price']).over(windowSpec2))

In [11]:
products.select(
    products['product'],
    products['category'],
    products['price'],
    price_max.alias("price_max")).show()

+----------+--------+-----+---------+
|   product|category|price|price_max|
+----------+--------+-----+---------+
|    iPhone|  Mobile|  999|      999|
|Samsung JX|  Mobile|  799|      999|
|Redmi Note|  Mobile|  399|      799|
|   OnePlus|  Mobile|  356|      399|
|        Mi|  Mobile|  299|      356|
|  Micromax|  Mobile|  249|      299|
|Samsung TX|  Tablet|  999|      999|
|      iPad|  Tablet|  789|      999|
|    Lenovo|  Tablet|  499|      789|
|        Xu|  Tablet|  267|      499|
+----------+--------+-----+---------+



#### Window price difference function between ranges

In [12]:
windowSpec3 = Window \
    .partitionBy(products['category']) \
    .orderBy(products['price'].desc()) \
    .rangeBetween(-sys.maxsize, sys.maxsize)

In [13]:
price_difference = \
  (func.max(products['price']).over(windowSpec3) - products['price'])

In [14]:
products.select(
    products['product'],
    products['category'],
    products['price'],
    price_difference.alias("price_difference")).show()

+----------+--------+-----+----------------+
|   product|category|price|price_difference|
+----------+--------+-----+----------------+
|    iPhone|  Mobile|  999|             0.0|
|Samsung JX|  Mobile|  799|           200.0|
|Redmi Note|  Mobile|  399|           600.0|
|   OnePlus|  Mobile|  356|           643.0|
|        Mi|  Mobile|  299|           700.0|
|  Micromax|  Mobile|  249|           750.0|
|Samsung TX|  Tablet|  999|             0.0|
|      iPad|  Tablet|  789|           210.0|
|    Lenovo|  Tablet|  499|           500.0|
|        Xu|  Tablet|  267|           732.0|
+----------+--------+-----+----------------+



In [104]:
windowSpec4 = Window \
    .partitionBy(products['category']) \
    .orderBy(products['price'].asc()) \
    .rangeBetween(0, sys.maxsize)

In [105]:
sys.maxsize

9223372036854775807

In [106]:
price_max = (func.max(products['price']).over(windowSpec4))

In [107]:
products.select(
    products['product'],
    products['category'],
    products['price'],
    price_max.alias("price_max")).show()

+----------+--------+-----+---------+
|   product|category|price|price_max|
+----------+--------+-----+---------+
|  Micromax|  Mobile|  249|      999|
|        Mi|  Mobile|  299|      999|
|   OnePlus|  Mobile|  356|      999|
|Redmi Note|  Mobile|  399|      999|
|Samsung JX|  Mobile|  799|      999|
|    iPhone|  Mobile|  999|      999|
|        Xu|  Tablet|  267|      999|
|    Lenovo|  Tablet|  499|      999|
|      iPad|  Tablet|  789|      999|
|Samsung TX|  Tablet|  999|      999|
+----------+--------+-----+---------+

